# 1. Import Libraries

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import time

# 2. Load Data

In [21]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dir = 'output_dataset/train'
val_dir = 'output_dataset/val'
test_dir = 'output_dataset/test'

train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Dataset sizes
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

# Data loaders dictionary
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

# 3. Define the Model

In [22]:
# Load pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer for your specific task
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)


# 4. Fine-tune the Model

In [23]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Initialize history dictionary to store metrics
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc * 100:.4f}')

            # Store in history
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())  # Convert tensor to float

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc * 100:.4f}')

    model.load_state_dict(best_model_wts)
    return model, history

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model, history = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)



Epoch 1/10
train Loss: 1.2309 Acc: 51.1905
val Loss: 2.0879 Acc: 45.5556

Epoch 2/10
train Loss: 0.8436 Acc: 68.3333
val Loss: 7.7481 Acc: 30.0000

Epoch 3/10
train Loss: 0.7612 Acc: 70.4762
val Loss: 3.0842 Acc: 32.2222

Epoch 4/10
train Loss: 0.7633 Acc: 68.8095
val Loss: 2.1591 Acc: 38.8889

Epoch 5/10
train Loss: 0.6998 Acc: 73.8095
val Loss: 1.9833 Acc: 55.5556

Epoch 6/10
train Loss: 0.5543 Acc: 78.0952
val Loss: 1.2153 Acc: 53.3333

Epoch 7/10
train Loss: 0.4563 Acc: 83.0952
val Loss: 2.5864 Acc: 65.5556

Epoch 8/10
train Loss: 0.3579 Acc: 85.7143
val Loss: 1.2975 Acc: 64.4444

Epoch 9/10
train Loss: 0.2854 Acc: 90.0000
val Loss: 1.1662 Acc: 65.5556

Epoch 10/10
train Loss: 0.2302 Acc: 92.3810
val Loss: 1.1336 Acc: 62.2222

Training complete in 16m 17s
Best val Acc: 65.5556


# 5. Save the Model

In [24]:
# Save the trained model
torch.save(model.state_dict(), 'resnet18_baseline.pth')

# 6. Prune the Model

### 6.1 Load pretrained baseline model

In [25]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pretrained ResNet18 with 4 output classes
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # Adjust for 4 classes
model.load_state_dict(torch.load('resnet18_baseline.pth'))  # Make sure you've saved baseline
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 6.2 Prunning configuration with 30% Weights pruned

In [26]:
import torch.nn.utils.prune as prune

def prune_and_remove(model, amount=0.3):
    """
    Prunes 30% of weights in all Conv2d and Linear layers, then makes it permanent.
    """
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            # Apply unstructured L1 pruning
            prune.l1_unstructured(module, name='weight', amount=amount)
            # Remove the pruning mask and make it permanent
            prune.remove(module, 'weight')
    return model

pruned_model = prune_and_remove(model, amount=0.3)

### 6.3 Train pruned model and save it

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pruned_model.parameters(), lr=0.0001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Train again
pruned_model, pruned_history = train_model(pruned_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

# Save pruned model
torch.save(pruned_model.state_dict(), 'resnet18_pruned.pth')



Epoch 1/10
train Loss: 0.3137 Acc: 88.5714
val Loss: 1.1355 Acc: 61.1111

Epoch 2/10
train Loss: 0.1591 Acc: 94.2857
val Loss: 1.1537 Acc: 62.2222

Epoch 3/10
train Loss: 0.0872 Acc: 97.3810
val Loss: 1.2453 Acc: 58.8889

Epoch 4/10
train Loss: 0.0804 Acc: 98.5714
val Loss: 1.3401 Acc: 62.2222

Epoch 5/10
train Loss: 0.0545 Acc: 98.5714
val Loss: 1.2387 Acc: 65.5556

Epoch 6/10
train Loss: 0.0341 Acc: 99.5238
val Loss: 1.2671 Acc: 61.1111

Epoch 7/10
train Loss: 0.0341 Acc: 99.5238
val Loss: 1.3026 Acc: 64.4444

Epoch 8/10
train Loss: 0.0323 Acc: 98.8095
val Loss: 1.2756 Acc: 64.4444

Epoch 9/10
train Loss: 0.0265 Acc: 99.7619
val Loss: 1.2860 Acc: 64.4444

Epoch 10/10
train Loss: 0.0291 Acc: 99.5238
val Loss: 1.3159 Acc: 64.4444

Training complete in 16m 60s
Best val Acc: 65.5556


# 7. Evaluate the Model

In [ ]:
# Define evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


# Evaluate baseline model
model.load_state_dict(torch.load('resnet18_baseline.pth'))
baseline_accuracy = evaluate_model(model, test_loader)
print(f'Baseline Model Accuracy: {baseline_accuracy:.2f}%')

# Evaluate pruned model
model.load_state_dict(torch.load('resnet18_pruned.pth'))
pruned_accuracy = evaluate_model(model, test_loader)
print(f'Pruned Model Accuracy: {pruned_accuracy:.2f}%')

Baseline Model Accuracy: 86.04%
Pruned Model Accuracy: 87.14%
